<a href="https://colab.research.google.com/github/IritaSee/detection-web/blob/main/Crawling_Data_News.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()


KeyboardInterrupt: 

In [3]:
!pip install requests beautifulsoup4 pandas pytrends

In [4]:
import json
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd

# Load JSON file
with open('news_websites.json') as json_file:
    websites = json.load(json_file)['websites']

# Function to scrape a single website
def scrape_website(website):
    url = website['url']
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    news_data = []
    for article in soup.find_all(website['title_tag'], class_=website['title_class']):
        title = article.get_text() if article else ''
        link_tag = article.find(website['link_tag'], class_=website['link_class'])
        link = link_tag['href'] if link_tag else ''
        date_tag = article.find(website['date_tag'], class_=website['date_class'])
        date_span = date_tag.find('span', class_=website['date_span_class']) if date_tag else None
        timestamp = int(date_span.get_text()) if date_span else 0
        date = datetime.fromtimestamp(timestamp).strftime('%d/%m/%Y') if timestamp else ''

        # Get the article content
        content = ''
        if link:
            article_response = requests.get(link)
            article_soup = BeautifulSoup(article_response.text, 'html.parser')
            content_div = article_soup.find('div', class_=website['content_div_class'])
            content = ' '.join(p.get_text() for p in content_div.find_all('p')) if content_div else ''

        news_data.append({
            'title': title,
            'link': link,
            'date': date,
            'content': content,
            'is_fake': 0,
            'media_bias': 'neutral'
        })

    return news_data

# Scrape all websites
all_news = []
for website in websites:
    all_news.extend(scrape_website(website))

# Convert to DataFrame
df = pd.DataFrame(all_news)

# Save to CSV
df.to_csv('scraped_news.csv', index=False)

# Display the DataFrame
df.head()


KeyError: 'title_tag'

In [2]:
def write_to_csv(data, filename):
    with open(filename, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        # Write header row
        writer.writerow(['title', 'link', 'date', 'content', 'is_fake', 'media_bias'])
        # Write data rows
        for row in data:
            writer.writerow(row)

#flow:
input topik -> crawl berdasarkan topik -> deteksi bahasa -> masukin ke AI -> average

In [43]:
!pip install newsapi-python

In [79]:
# prompt: get request and print the result, newline every element in the json

import requests
api_key = "5fa9f96725e54435abf722862b9e4970"

# Define the URL to send the GET request to
# url = ('https://newsapi.org/v2/everything?'
#        'q=apple&'
#        'from=2024-05-25&'
#        'to=2024-06-01&'
#       #  'language=fr&'
#        'apiKey='+api_key)
url = "https://newsapi.org/v2/top-headlines?q=papua&country=id&apiKey="+api_key

response = requests.get(url)

# Check for successful response
if response.status_code == 200:
    # Parse the JSON data
    data = response.json()

    # Print each element in the JSON data, separated by a newline
    for element in data:
        print(element)
        print(data[str(element)])
        print()
else:
    # Print an error message
    print(f"Error: {response.status_code}")


status
ok

totalResults
1

articles
[{'source': {'id': 'google-news', 'name': 'Google News'}, 'author': 'Radar Papua', 'title': 'Penemuan Mengejutkan! Tembok Bawah Laut yang Mengelilingi Bumi Ditemukan di Google Earth - Radar Papua - Radar Papua', 'description': None, 'url': 'https://news.google.com/rss/articles/CBMilAFodHRwczovL3JhZGFycGFwdWEuamF3YXBvcy5jb20vbGlmZXN0eWxlLXRla25vbG9naS8zMjQ4MzI2MTIvcGVuZW11YW4tbWVuZ2VqdXRrYW4tdGVtYm9rLWJhd2FoLWxhdXQteWFuZy1tZW5nZWxpbGluZ2ktYnVtaS1kaXRlbXVrYW4tZGktZ29vZ2xlLWVhcnRo0gGYAWh0dHBzOi8vcmFkYXJwYXB1YS5qYXdhcG9zLmNvbS9saWZlc3R5bGUtdGVrbm9sb2dpL2FtcC8zMjQ4MzI2MTIvcGVuZW11YW4tbWVuZ2VqdXRrYW4tdGVtYm9rLWJhd2FoLWxhdXQteWFuZy1tZW5nZWxpbGluZ2ktYnVtaS1kaXRlbXVrYW4tZGktZ29vZ2xlLWVhcnRo?oc=5', 'urlToImage': None, 'publishedAt': '2024-07-05T14:31:31Z', 'content': None}]



In [82]:
from newsapi import NewsApiClient

# Init
newsapi = NewsApiClient(api_key='5fa9f96725e54435abf722862b9e4970')

# # /v2/top-headlines
top_headlines = newsapi.get_top_headlines(# q='bitcoin',
                                          # sources='bbc-news,the-verge',
                                          # category='business',
                                          # language='en',
                                          country = 'id')

# /v2/everything
# all_articles = newsapi.get_everything(q='bitcoin',
#                                       # sources='bbc-news,the-verge',
#                                       # domains='bbc.co.uk,techcrunch.com',
#                                       # from_param='2024-06-05',
#                                       # to='2024-06-06',
#                                       language='en',
#                                       sort_by='relevancy',
#                                       page=2)

all_articles = newsapi.get_everything(q='heru budi',
                                      # sources='bbc-news,the-verge',
                                      # domains='en.tempo.co',
                                      # from_param='2024-06-05',
                                      # to='2024-06-06',
                                      # language='en',
                                      # sort_by='relevancy',
                                      # page=2
                                      )

# /v2/top-headlines/sources
sources = newsapi.get_sources()

In [83]:
all_articles

{'status': 'ok',
 'totalResults': 12,
 'articles': [{'source': {'id': None, 'name': 'Suara.com'},
   'author': 'Bimo Aria Fundrika',
   'title': 'Baru Ulang Tahun ke-497, Kualitas Udara Jakarta Malah Terburuk Ketiga di Dunia, Guru Besar FKUI Beri Saran Begini',
   'description': 'Indeks kualitas udara (AQI) di Jakarta berada di angka 166, dengan angka partikel halus (particulate matter/PM) 2,5 di angka konsentrasi 77 mikrogram per meter kubik.',
   'url': 'https://www.suara.com/health/2024/06/23/140919/baru-ulang-tahun-ke-497-kualitas-udara-jakarta-malah-terburuk-ketiga-di-dunia-guru-besar-fkui-beri-saran-begini',
   'urlToImage': 'https://media.suara.com/pictures/970x544/2022/06/15/71271-polusi-jakarta.jpg',
   'publishedAt': '2024-06-23T07:16:18Z',
   'content': 'Suara.com - Jakarta baru saja ulang tahun ke-497 pada 22 Juni kemarin. Namun di tengah kegembiraan itu, Jakarta masih diselimuti persoalan klasik tentang polusi udara. Bahkan saat Minggu pagi, di man… [+3536 chars]'},
  {'so

##pake codingan buatan sendiri tapi bisa search:

In [86]:
import json
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd
import time

search_query = "heru budi"
search_query = search_query.replace(" ", "+")

# Load JSON file
with open('news_websites_with_query.json') as json_file:
    websites = json.load(json_file)['websites']

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# Function to scrape a single website
def scrape_website(website):
    url = website['url'] + search_query
    print(f"Scraping {website['name']} at {url}")
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')

    news_data = []
    articles = soup.find_all(website['article_tag'], class_=website['article_class'])
    print(f"Found {len(articles)} articles on {website['name']}")

    for article in articles:
        title_tag = article.find(class_=website.get('title_class', None))
        if title_tag:
            title = title_tag.get_text(strip=True)
        else:
            # Fallback to find title using tag and attribute
            title_tag = article.find('a')
            title = title_tag['title'] if 'title' in title_tag.attrs else title_tag.get_text(strip=True)

        link_tag = article.find('a')
        link = link_tag['href'] if link_tag else 'No link found'

        date_tag = article.find(class_=website['date_class'])
        date = date_tag.get_text(strip=True) if date_tag else 'No date found'

        # Get the article content
        content = ''
        if link != 'No link found':
            article_response = requests.get(link, headers=headers)
            article_response.raise_for_status()
            article_soup = BeautifulSoup(article_response.text, 'html.parser')
            content_div = article_soup.find(class_=website['content_class'])
            content = ' '.join(p.get_text(strip=True) for p in content_div.find_all('p')) if content_div else 'No content found'

        news_data.append({
            'title': title,
            'link': link,
            'date': date,
            'content': content,
            'is_fake': 0,
            'media_bias': website['platform']
        })
        print(f"Appended article: {title}")

    return news_data

# Scrape all websites
all_news = []
for website in websites:
    try:
        scraped_news = scrape_website(website)
        print(f"Scraped {len(scraped_news)} articles from {website['name']}")
        all_news.extend(scraped_news)
        time.sleep(2)  # Respectful delay to avoid overwhelming the server
    except requests.HTTPError as e:
        print(f"Failed to scrape {website['name']}: {e}")

# Convert to DataFrame
df = pd.DataFrame(all_news)
print(f"Total articles collected: {len(all_news)}")

# Save to CSV
df.to_csv('scraped_news.csv', index=False)

# Display the DataFrame
df.head()


Scraping CNN Indonesia at https://www.cnnindonesia.com/search/?query=heru+budi
Found 0 articles on CNN Indonesia
Scraped 0 articles from CNN Indonesia
Total articles collected: 0


""


#flow:
crawl ke berbagai situs berita -> AI sentimen -> tentukan "vibe" kator berita itu pada hari itu

In [93]:
import json
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd
import time

# Load JSON file
with open('news_websites.json') as json_file:
    websites = json.load(json_file)['websites']

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# Function to scrape a single website
def scrape_website(website):
    url = website['url']
    print(f"Scraping {website['name']} at {url}")
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')

    news_data = []
    articles = soup.find_all(website['article_tag'], class_=website['article_class'])
    print(f"Found {len(articles)} articles on {website['name']}")

    for article in articles:
        title_tag = article.find(class_=website.get('title_class', None))
        if title_tag:
            title = title_tag.get_text(strip=True)
        else:
            # Fallback to find title using tag and attribute
            title_tag = article.find('a')
            title = title_tag['title'] if 'title' in title_tag.attrs else title_tag.get_text(strip=True)

        link_tag = article.find('a')
        link = link_tag['href'] if link_tag else 'No link found'

        date_tag = article.find(class_=website['date_class'])
        date = date_tag.get_text(strip=True) if date_tag else 'No date found'

        # Get the article content
        content = ''
        if link != 'No link found':
            article_response = requests.get(link, headers=headers)
            article_response.raise_for_status()
            article_soup = BeautifulSoup(article_response.text, 'html.parser')
            content_div = article_soup.find(class_=website['content_class'])
            content = ' '.join(p.get_text(strip=True) for p in content_div.find_all('p')) if content_div else 'No content found'

        news_data.append({
            'title': title,
            'link': link,
            'date': date,
            'content': content,
            'is_fake': 0,
            'media_bias': website['platform']
        })
        print(f"Appended article: {title}")

    return news_data

# Scrape all websites
all_news = []
for website in websites:
    try:
        scraped_news = scrape_website(website)
        print(f"Scraped {len(scraped_news)} articles from {website['name']}")
        all_news.extend(scraped_news)
        time.sleep(2)  # Respectful delay to avoid overwhelming the server
    except requests.HTTPError as e:
        print(f"Failed to scrape {website['name']}: {e}")

# Convert to DataFrame
df = pd.DataFrame(all_news)
print(f"Total articles collected: {len(all_news)}")

# Save to CSV
df.to_csv('scraped_news.csv', index=False)

# Display the DataFrame
df.head()


Scraping CNN Indonesia at https://www.cnnindonesia.com/nasional
Found 0 articles on CNN Indonesia
Scraped 0 articles from CNN Indonesia
Scraping Detik at https://news.detik.com/berita/indeks/
Found 0 articles on Detik
Scraped 0 articles from Detik
Scraping Kompas at https://www.kompas.com/tag/politik
Found 15 articles on Kompas
Appended article: Megawati Akhirnya Sebut Nama Jokowi Usai "Huru-hara" Pilpres, Singgung Hubungan dan Utang Negara
Appended article: Alasan KPU Tak Minta Maaf Usai Ketua KPU Hasyim Langgar Etik
Appended article: Heru Budi Bisa Saja Ikut Pilkada, tapi Dianggap Sulit Diusung Partai Politik
Appended article: Wapres: Isu Palestina Bukan soal Agama, tapi Politik dan Kemanusiaan
Appended article: Bobby Nasution Ungkap Peran Jokowi dalam Pencalonannya di Pilkada Sumut, Apa Itu?
Appended article: Survei Indikator Politik: Kans Dedi Mulyadi jika Ridwan Kamil Ikut Pilkada Jakarta
Appended article: Survei Indikator Politik: Elektabilitas Ridwan Kamil, Dedi Mulyadi, dan Kom

,title,link,date,content,is_fake,media_bias
0,"Megawati Akhirnya Sebut Nama Jokowi Usai ""Huru...",https://video.kompas.com/watch/1563709/megawat...,"05/07/2024, 17:21 WIB",Ketua Umum PDI-P Megawati Soekarnoputri untuk ...,0,kompas
1,Alasan KPU Tak Minta Maaf Usai Ketua KPU Hasyi...,https://video.kompas.com/watch/1563430/alasan-...,"05/07/2024, 15:38 WIB",Komisioner KPU August Mellaz mengungkapkan ala...,0,kompas
2,"Heru Budi Bisa Saja Ikut Pilkada, tapi Diangga...",http://megapolitan.kompas.com/read/2024/07/05/...,"05/07/2024, 06:47 WIB","JAKARTA, KOMPAS.com- Penjabat Gubernur (Pj) Ja...",0,kompas
3,"Wapres: Isu Palestina Bukan soal Agama, tapi P...",http://nasional.kompas.com/read/2024/07/05/063...,"05/07/2024, 06:35 WIB","JAKARTA, KOMPAS.com– Wakil Presiden (Wapres) R...",0,kompas
4,Bobby Nasution Ungkap Peran Jokowi dalam Penca...,https://video.kompas.com/watch/1561807/bobby-n...,"04/07/2024, 20:54 WIB",Wali Kota Medan Bobby Nasution mengungkapkan p...,0,kompas


#AI part

In [87]:
import os
import pickle
import tensorflow as tf
from keras.preprocessing.sequence import pad_sequences


tokenizer_A3_path = os.path.join( 'model', 'tokenizer_A3.pkl')
tokenizer_bias_path = os.path.join( 'model', 'tokenizer_bias.pkl')

model_A3_path = os.path.join( 'model', 'hoax_detection_A3.tflite')
model_bias_path = os.path.join('model', 'bias_detection.tflite')

# Load tokenizer from pickle file
with open(tokenizer_A3_path, 'rb') as f:
    tokenizer_A3 = pickle.load(f)

with open(tokenizer_bias_path, 'rb') as f:
    tokenizer_bias = pickle.load(f)

# Load TFLite model
interpreterHoaks = tf.lite.Interpreter(model_path=model_A3_path)
interpreterHoaks.allocate_tensors()

interpreterBias = tf.lite.Interpreter(model_path=model_bias_path)
interpreterBias.allocate_tensors()

# Get input and output tensor information
input_details_hoaks = interpreterHoaks.get_input_details()
output_details_hoaks = interpreterHoaks.get_output_details()

input_details_bias = interpreterBias.get_input_details()
output_details_bias = interpreterBias.get_output_details()

In [88]:
def predictBias(news_text1, news_text2):

    # Combine input texts
    news_text = [news_text1 + " " + news_text2]

    # Tokenization and padding of news
    new_sequences = tokenizer_bias.texts_to_sequences(news_text)
    max_len = 30  # Make sure the maximum length matches the one used when training the model
    new_padded = pad_sequences(new_sequences, maxlen=max_len)

    # Convert input data to float32 type
    new_padded = new_padded.astype('float32')

    # Set the input tensor with compacted data
    interpreterBias.set_tensor(input_details_bias[0]['index'], new_padded)

    # Run the interpreter to make predictions
    interpreterBias.invoke()

    # Get the prediction result from the output tensor
    predictions_tflite = interpreterBias.get_tensor(output_details_bias[0]['index'])

    # Interpreting prediction results
    predicted_labels_tflite = "Bias" if predictions_tflite[0][0] > 0.5 else "Netral"
    # confidence = float(predictions_tflite[0][0]) if predicted_labels_tflite == "Hoax" else float(1 - predictions_tflite[0][0])
    confidence = float(predictions_tflite[0][0])

    # Prepare response
    response = {
        'prediction': predicted_labels_tflite,
        'confidence': confidence
    }

    return response

In [100]:
df['content'][9]

'JAKARTA, KOMPAS.com- Jajak pendapat yang diselenggarakanIndikator Politik Indonesiapada 20-27 Juni 2024 menunjukkan, mantan Gubernur Jawa BaratRidwan Kamilunggul dalam seluruh simulasi survei elektabilitas calon gubernur Jawa Barat. Pendiri Indikator Politik IndonesiaBurhanuddin Muhtadimenyebutkan, dalam simulasitop of mind, Ridwan Kamil unggul dengan elektabilitas 16 persen, begitu pula dalam simulasi 26 nama dengan keterpilihan 36,8 persen. "Hasilnya kurang lebih dengan pola ditop of mind, nomor satu Pak Ridwan Kamil di survei kita juga menemukan hal yang sama dalam simulasi semi terbuka, (di simulasi 26 nama) 36,8 persen. Peringkat keduaDedi Mulyadi31,9," kata Burhanuddin dalam konferensi pers daring, Kamis (4/7/2024). Baca juga:Bersaing Ketat dengan Ridwan Kamil di Pilkada Jabar, Dedi Mulyadi: Mohon Doanya Selanjutnya, dalam simulasi 12 nama, Ridwan Kamil mendapat elektabilitas 44,5 persen, simulasi 6 nama 45,6 persen, dan simulasi 3 nama 56,3 persen. Dedi Mulyadi juga konsisten b

In [101]:
print(predictBias(df['content'][9], df['link'][9]))

{'prediction': 'Bias', 'confidence': 0.6660130023956299}
